# ETL

<a target="_blank" href="https://colab.research.google.com/github/vitostamatti/intro-python/blob/main/notebooks/09-etl.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## Extract


In [9]:
# pip install azure-storage-blob azure-identity
import os
from azure.storage.blob import BlobServiceClient, BlobClient
import pandas as pd
import io

In [10]:
blob_service_client = BlobServiceClient.from_connection_string(
    "DefaultEndpointsProtocol=https;AccountName=smartsappstorage;AccountKey=MsCRSvFZgXVZyDWudau6a17pItN+D8UDJ8nNPAQ7uq5AZ8p62PrvUki3HaK84kaI2iCIWyYPGBUWB0FWGT4WUw==;EndpointSuffix=core.windows.net"
)

In [11]:
list(blob_service_client.list_containers())

[{'name': 'azure-webjobs-hosts', 'last_modified': datetime.datetime(2017, 9, 20, 11, 13, 48, tzinfo=datetime.timezone.utc), 'etag': '"0x8D50018AA6E2B87"', 'lease': {'status': 'unlocked', 'state': 'available', 'duration': None}, 'public_access': None, 'has_immutability_policy': False, 'deleted': None, 'version': None, 'has_legal_hold': False, 'metadata': None, 'encryption_scope': <azure.storage.blob._models.ContainerEncryptionScope object at 0x0000020E718C5480>, 'immutable_storage_with_versioning_enabled': False},
 {'name': 'azure-webjobs-secrets', 'last_modified': datetime.datetime(2019, 1, 15, 15, 31, 52, tzinfo=datetime.timezone.utc), 'etag': '"0x8D67AFE92ECC01E"', 'lease': {'status': 'unlocked', 'state': 'available', 'duration': None}, 'public_access': None, 'has_immutability_policy': False, 'deleted': None, 'version': None, 'has_legal_hold': False, 'metadata': None, 'encryption_scope': <azure.storage.blob._models.ContainerEncryptionScope object at 0x0000020E718C51B0>, 'immutable_st

In [12]:
container_name = "datos-auctioninsights"

container_client = blob_service_client.get_container_client(container=container_name)

list(container_client.list_blobs())

[{'name': 'Historical Data/Impresiones(MX)_04042023.csv', 'container': 'datos-auctioninsights', 'snapshot': None, 'version_id': None, 'is_current_version': None, 'blob_type': <BlobType.BLOCKBLOB: 'BlockBlob'>, 'metadata': {}, 'encrypted_metadata': None, 'last_modified': datetime.datetime(2023, 4, 10, 13, 24, 40, tzinfo=datetime.timezone.utc), 'etag': '0x8DB39C6F0CE74CF', 'size': 2326345, 'content_range': None, 'append_blob_committed_block_count': None, 'is_append_blob_sealed': None, 'page_blob_sequence_number': None, 'server_encrypted': True, 'copy': {'id': None, 'source': None, 'status': None, 'progress': None, 'completion_time': None, 'status_description': None, 'incremental_copy': None, 'destination_snapshot': None}, 'content_settings': {'content_type': 'application/octet-stream', 'content_encoding': None, 'content_language': None, 'content_md5': None, 'content_disposition': None, 'cache_control': None}, 'lease': {'status': 'unlocked', 'state': 'available', 'duration': None}, 'blob_

In [13]:
keywords_blobs = "Historical Data/PalabrasClave"
impressions_blobs = "Historical Data/Impresiones"

keyword_blob_to_process = container_client.walk_blobs(
    name_starts_with=keywords_blobs
).next()
for b in container_client.walk_blobs(name_starts_with=keywords_blobs):
    if b.creation_time > keyword_blob_to_process.creation_time:
        keyword_blob_to_process = b

impressions_blob_to_process = container_client.walk_blobs(
    name_starts_with=impressions_blobs
).next()
for b in container_client.walk_blobs(name_starts_with=impressions_blobs):
    if b.creation_time > impressions_blob_to_process.creation_time:
        impressions_blob_to_process = b

In [27]:
impressions_blob_to_process["name"], keyword_blob_to_process["name"]

('Historical Data/Impresiones_MX.csv', 'Historical Data/PalabrasClave_MX.csv')

In [28]:
blob_client = blob_service_client.get_blob_client(
    container=container_name, blob=keyword_blob_to_process
)
content = blob_client.download_blob().content_as_text()

In [29]:
keywords_df = pd.read_csv(io.StringIO(content))

In [30]:
# saving is optional
keywords_df.to_csv("../data/etl/keywords.csv", index=False)

In [31]:
keywords_df.columns

Index(['Month', 'Account name', 'Customer ID', 'Campaign', 'Campaign ID',
       'Ad group', 'Ad group ID', 'Search keyword', 'Keyword ID',
       'Display URL domain', 'Search Impr. share (Auction Insights)',
       'Search overlap rate', 'Position above rate', 'Top of page rate',
       'Abs. Top of page rate', 'Search outranking share'],
      dtype='object')

In [32]:
keywords_df.shape

(81183, 16)

In [33]:
keywords_df.head()

,Month,Account name,Customer ID,Campaign,Campaign ID,Ad group,Ad group ID,Search keyword,Keyword ID,Display URL domain,Search Impr. share (Auction Insights),Search overlap rate,Position above rate,Top of page rate,Abs. Top of page rate,Search outranking share
0,March 2023,AN~Bimbo_PR~Coronado_AV~FB_MK~MX_AR~STA,607-473-2610,031911/2023 - 000_Marzo_Ricolino_Untables_Coro...,16249142822,CORONADO - RECETAS,136169411889,ingredientes cajeta coronado,1393455534473,You,64.29%,--,--,100.00%,100.00%,--
1,February 2023,AN~Bimbo_PR~Coronado_AV~FB_MK~MX_AR~STA,607-473-2610,031911/2023 - 000_Marzo_Ricolino_Untables_Coro...,16249142822,CORONADO - COMPETENCIA,136169411729,nutella,19556113,amazon.com.mx,12.82%,39.05%,12.46%,93.14%,47.36%,11.91%
2,April 2023,AN~Bimbo_PR~Coronado_AV~FB_MK~MX_AR~STA,607-473-2610,031911/2023 - 000_Marzo_Ricolino_Untables_Coro...,16249142822,CORONADO - COMPETENCIA,136169411729,cajeta tillot,1393455534153,You,22.68%,--,--,90.10%,70.63%,--
3,April 2023,AN~Abbott_PR~SEM-Glucerna_AV~HM_MK~MX_AR~SPRK,333-992-4476,010388/2022_Glucerna E-commerce,10165198298,Bebible,148076042307,glucerna 237 ml,385743183801,You,47.97%,--,--,87.32%,78.87%,--
4,January 2023,AN~Abbott_PR~SEM-Glucerna_AV~HM_MK~MX_AR~SPRK,333-992-4476,010388/2022_Glucerna E-commerce,10165198298,Bebible,148076042307,glucerna bebible precio,1672684181430,amazon.com.mx,100.00%,100.00%,0.00%,87.50%,25.00%,75.00%


In [34]:
keywords_df["Keyword ID"].nunique()

8429

In [35]:
blob_client = blob_service_client.get_blob_client(
    container=container_name, blob=impressions_blob_to_process
)
imp_content = blob_client.download_blob().content_as_text()

In [36]:
impressions_df = pd.read_csv(io.StringIO(imp_content))

In [37]:
impressions_df.to_csv("../data/etl/impressions.csv", index=False)

In [38]:
impressions_df.columns

Index(['Month', 'Account name', 'Customer ID', 'Campaign', 'Campaign ID',
       'Ad group', 'Ad group ID', 'Search keyword', 'Keyword ID', 'Impr.'],
      dtype='object')

In [39]:
impressions_df.shape

(31193, 10)

In [40]:
impressions_df.head()

,Month,Account name,Customer ID,Campaign,Campaign ID,Ad group,Ad group ID,Search keyword,Keyword ID,Impr.
0,March 2023,AN~Bimbo_PR~Coronado_AV~FB_MK~MX_AR~STA,607-473-2610,031911/2023 - 000_Marzo_Ricolino_Untables_Coro...,16249136333,RECETAS,128609968930,postres con leche condensada,304019792612,400
1,March 2023,AN~Bimbo_PR~Coronado_AV~FB_MK~MX_AR~STA,607-473-2610,031911/2023 - 000_Marzo_Ricolino_Untables_Coro...,16249136333,RECETAS,128609968930,recetas rapidas,141469255,"6,285"
2,April 2023,AN~Bimbo_PR~Coronado_AV~FB_MK~MX_AR~STA,607-473-2610,031911/2023 - 000_Marzo_Ricolino_Untables_Coro...,16249136333,RECETAS,128609968930,ideas para postres,385359267764,"1,958"
3,March 2023,AN~Bimbo_PR~Coronado_AV~FB_MK~MX_AR~STA,607-473-2610,031911/2023 - 000_Marzo_Ricolino_Untables_Coro...,16249136333,RECETAS,128609968930,recetas de dulces fáciles de hacer en casa,1131278898409,32
4,April 2023,AN~Bimbo_PR~Coronado_AV~FB_MK~MX_AR~STA,607-473-2610,031911/2023 - 000_Marzo_Ricolino_Untables_Coro...,16249136333,POSTRES,128609969010,receta de pastel de tres leches,1972863163,"2,144"


In [41]:
impressions_df["Keyword ID"].nunique()

12024

## Transform


In [42]:
merged_df = keywords_df.merge(
    impressions_df[["Ad group ID", "Keyword ID", "Impr."]],
    left_on=["Ad group ID", "Keyword ID"],
    right_on=["Ad group ID", "Keyword ID"],
)

In [115]:
# accounts table
accounts = merged_df[["Account name", "Customer ID"]].drop_duplicates()
# campaigns table
campaigns = merged_df[["Campaign", "Campaign ID"]].drop_duplicates()
# adgroup table
adgroups = merged_df[["Ad group", "Ad group ID"]].drop_duplicates()
# keywords table
keywords = merged_df[["Search keyword", "Keyword ID"]].drop_duplicates()

In [116]:
merged_df.columns
cols = [
    "Month",
    "Customer ID",
    "Campaign ID",
    "Ad group ID",
    "Keyword ID",
    "Display URL domain",
    "Search Impr. share (Auction Insights)",
    "Search overlap rate",
    "Position above rate",
    "Top of page rate",
    "Abs. Top of page rate",
    "Search outranking share",
    "Impr.",
]

metrics = merged_df[cols]

In [117]:
# column renaming for convention
def normalize_column_names(data):
    cols = data.columns.to_list()

    data.columns = ["_".join(c.lower().replace(".", "").split(" ")) for c in cols]
    return data


accounts = normalize_column_names(accounts)
campaigns = normalize_column_names(campaigns)
adgroups = normalize_column_names(adgroups)
keywords = normalize_column_names(keywords)
metrics = normalize_column_names(metrics)

In [229]:
def transform_accounts(accounts):
    accounts = accounts.rename(columns={"customer_id": "account_id"})
    accounts["account_id"] = accounts.account_id.astype(str).str.replace("-", "")
    accounts["account_id"] = accounts["account_id"].astype(pd.Int64Dtype())
    return accounts


def transform_campaigns(campaigns):
    campaigns = campaigns.rename(columns={"campaign": "campaign_name"})
    campaigns["campaign_id"] = campaigns["campaign_id"].astype(pd.Int64Dtype())
    return campaigns


def transform_adgroups(adgroups):
    adgroups = adgroups.rename(columns={"ad_group": "ad_group_name"})
    adgroups["ad_group_id"] = adgroups["ad_group_id"].astype(pd.Int64Dtype())
    return adgroups


def transform_keywords(keywords):
    keywords = keywords.rename(columns={"search_keyword": "keyword_name"})
    keywords["keyword_id"] = keywords["keyword_id"].astype(pd.Int64Dtype())
    return keywords


def _convert_metrics_date(metrics):
    metrics.loc[:, "month"] = metrics.loc[:, "month"].str.lower()

    month_names = {
        "january": 1,
        "february": 2,
        "march": 3,
        "april": 4,
        "may": 5,
        "june": 6,
        "july": 7,
        "august": 8,
        "september": 9,
        "october": 10,
        "november": 11,
        "december": 12,
    }
    metrics.loc[:, "_month"] = (
        metrics.loc[:, "month"]
        .str.split(" ", expand=True)
        .drop(columns=[1])[0]
        .map(month_names)
    )
    metrics.loc[:, "_year"] = (
        metrics.loc[:, "month"].str.split(" ", expand=True).drop(columns=[0])[1]
    )
    metrics.loc[:, "date"] = pd.to_datetime(
        metrics.loc[:, "_year"].astype(str)
        + "-"
        + metrics.loc[:, "_month"].astype(str)
        + "-"
        + "1"
    )
    metrics = metrics.drop(columns=["_month", "_year"])

    return metrics


def _convert_percentage_metric_to_float(metric):
    return (
        float(str(metric).replace("<", "").replace("%", "").replace("--", "0").strip())
        / 100
    )


def _convert_int_metric(metric):
    return int(str(metric).replace(",", "").replace("--", "0").strip())


def transform_metrics(metrics):
    metrics = _convert_metrics_date(metrics)

    metrics = metrics.rename(
        columns={"search_impr_share_(auction_insights)": "search_impr_share"}
    )

    metrics.loc[:, "search_impr_share"] = (
        metrics.loc[:, "search_impr_share"]
        .apply(lambda x: _convert_percentage_metric_to_float(x))
        .astype(float)
    )
    metrics.loc[:, "search_overlap_rate"] = (
        metrics.loc[:, "search_overlap_rate"]
        .apply(lambda x: _convert_percentage_metric_to_float(x))
        .astype(float)
    )
    metrics.loc[:, "position_above_rate"] = (
        metrics.loc[:, "position_above_rate"]
        .apply(lambda x: _convert_percentage_metric_to_float(x))
        .astype(float)
    )
    metrics.loc[:, "top_of_page_rate"] = (
        metrics.loc[:, "top_of_page_rate"]
        .apply(lambda x: _convert_percentage_metric_to_float(x))
        .astype(float)
    )
    metrics.loc[:, "abs_top_of_page_rate"] = (
        metrics.loc[:, "abs_top_of_page_rate"]
        .apply(lambda x: _convert_percentage_metric_to_float(x))
        .astype(float)
    )
    metrics.loc[:, "search_outranking_share"] = (
        metrics.loc[:, "search_outranking_share"]
        .apply(lambda x: _convert_percentage_metric_to_float(x))
        .astype(float)
    )
    metrics.loc[:, "impr"] = (
        metrics.loc[:, "impr"].apply(lambda x: _convert_int_metric(x)).astype(int)
    )
    return metrics


accounts = transform_accounts(accounts)
campaigns = transform_campaigns(campaigns)
adgroups = transform_adgroups(adgroups)
keywords = transform_keywords(keywords)
metrics = transform_metrics(metrics)

In [230]:
accounts.dtypes

account_name    object
account_id       Int64
dtype: object

## Load


### Create DB Schema (if not exists)


In [241]:
from sqlalchemy import MetaData, PrimaryKeyConstraint
from sqlalchemy import (
    Table,
    Column,
    Integer,
    String,
    ForeignKey,
    BigInteger,
    DateTime,
    Float,
)
from sqlalchemy.engine import create_engine


engine = create_engine("sqlite:///:memory:")

metadata_obj = MetaData()

accounts_table = Table(
    "accounts",
    metadata_obj,
    Column("account_id", BigInteger, primary_key=True),
    Column("account_name", String(255), nullable=False),
)
campaigns_table = Table(
    "campaigns",
    metadata_obj,
    Column("campaign_id", BigInteger, primary_key=True),
    Column("campaign_name", String(255), nullable=False),
)
adgroups_table = Table(
    "adgroups",
    metadata_obj,
    Column("ad_group_id", BigInteger, primary_key=True),
    Column("ad_group_name", String(255), nullable=False),
)
keywords_table = Table(
    "keywords",
    metadata_obj,
    Column("keyword_id", BigInteger, primary_key=True),
    Column("keyword_name", String(255), nullable=False),
)

metrics_table = Table(
    "metrics",
    metadata_obj,
    Column(
        "account_id",
        BigInteger,
        ForeignKey("accounts.account_id"),
        nullable=False,
        # primary_key=True,
    ),
    Column(
        "campaign_id",
        BigInteger,
        ForeignKey("campaigns.campaign_id"),
        nullable=False,
        # primary_key=True,
    ),
    Column(
        "ad_group_id",
        BigInteger,
        ForeignKey("adgroups.ad_group_id"),
        nullable=False,
        # primary_key=True,
    ),
    Column(
        "keyword_id",
        BigInteger,
        ForeignKey("keywords.keyword_id"),
        nullable=False,
        # primary_key=True,
    ),
    Column("date", DateTime),
    Column("display_url_domain", String(255)),
    Column("search_impr_share", Float),
    Column("search_overlap_rate", Float),
    Column("position_above_rate", Float),
    Column("top_of_page_rate", Float),
    Column("abs_top_of_page_rate", Float),
    Column("search_outranking_share", Float),
    Column("impr", Integer),
    PrimaryKeyConstraint(
        "account_id", "campaign_id", "ad_group_id", "keyword_id", "date", name="pk"
    ),
)

metadata_obj.create_all(engine)

In [242]:
# metadata_obj.tables
import pandas as pd

q = """
SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';
"""

tables = pd.read_sql(q, engine)
tables

,name
0,accounts
1,campaigns
2,adgroups
3,keywords
4,metrics


In [243]:
for table in tables.name:
    q = f"""
    SELECT 
        *
    FROM 
        {table}
    """
    t = pd.read_sql(q, engine)
    print(table, t.columns.to_list())

accounts ['account_id', 'account_name']
campaigns ['campaign_id', 'campaign_name']
adgroups ['ad_group_id', 'ad_group_name']
keywords ['keyword_id', 'keyword_name']
metrics ['account_id', 'campaign_id', 'ad_group_id', 'keyword_id', 'date', 'display_url_domain', 'search_impr_share', 'search_overlap_rate', 'position_above_rate', 'top_of_page_rate', 'abs_top_of_page_rate', 'search_outranking_share', 'impr']


In [274]:
def update_table(engine, table, data):
    try:
        q = f"""SELECT *FROM {table.name}"""
        cur_data = pd.read_sql(q, engine)
        new_data = pd.concat([cur_data, data], axis=0)
        new_data = new_data[
            ~new_data.duplicated(
                subset=[c.name for c in table.primary_key.columns], keep=False
            )
        ]

        n_updates = new_data.to_sql(
            name=table.name, con=engine, if_exists="append", index=False
        )
        print(f"{n_updates if n_updates is not None else 0} rows inserted")
    except Exception as e:
        raise Exception(f"error updating database: {e}")

In [276]:
update_table(engine, table=accounts_table, data=accounts)
update_table(engine, table=campaigns_table, data=campaigns)
update_table(engine, table=adgroups_table, data=adgroups)
update_table(engine, table=keywords_table, data=keywords)

0 rows inserted
0 rows inserted
0 rows inserted
0 rows inserted


In [282]:
for table in tables.name:
    q = f"""
    SELECT 
         *
    FROM 
        {table}
    LIMIT 5
    """
    t = pd.read_sql(q, engine)
    print(table)
    print(t)
    print()

accounts
   account_id                                    account_name
0  6074732610         AN~Bimbo_PR~Coronado_AV~FB_MK~MX_AR~STA
1  3339924476   AN~Abbott_PR~SEM-Glucerna_AV~HM_MK~MX_AR~SPRK
2  7236167996  AN~Abbott_PR~SEM-Pediasure_AV~HM_MK~MX_AR~SPRK
3  5259784445     AN~Abbott_PR~SEM-Ensure_AV~HM_MK~MX_AR~SPRK
4  2956535590    AN~Abbott_PR~SEM-Similac_AV~HM_MK~MX_AR~SPRK

campaigns
   campaign_id                                      campaign_name
0  16249142822  031911/2023 - 000_Marzo_Ricolino_Untables_Coro...
1  10165198298                    010388/2022_Glucerna E-commerce
2  10173828666              120711/2021/2022_Pediasure E-commerce
3  10173766488               120711/2021/2022_Pediasure Awareness
4  16249136333  031911/2023 - 000_Marzo_Ricolino_Untables_Coro...

adgroups
    ad_group_id           ad_group_name
0  136169411889      CORONADO - RECETAS
1  136169411729  CORONADO - COMPETENCIA
2  148076042307                 Bebible
3  148076042107                   Brand
4 